In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

train_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 7874 images belonging to 8 classes.
Found 1691 images belonging to 8 classes.
Found 1687 images belonging to 8 classes.


In [4]:
# ResNet50 모델 불러오기
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# 모델 커스터마이징
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # 새로운 FC 레이어
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # 새로운 softmax 레이어

# 전체 모델 설정
model = Model(inputs=base_model.input, outputs=predictions)

# 첫째로: (랜덤 초기화된) 새로운 레이어들만 학습
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=13,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

model.summary()

Epoch 1/13
246/246 [==============================] - 36s 133ms/step - loss: 2.0433 - accuracy: 0.1885 - val_loss: 2.1391 - val_accuracy: 0.1292
Epoch 2/13
246/246 [==============================] - 25s 101ms/step - loss: 1.7972 - accuracy: 0.2641 - val_loss: 2.1215 - val_accuracy: 0.1286
Epoch 3/13
246/246 [==============================] - 22s 91ms/step - loss: 1.5867 - accuracy: 0.3514 - val_loss: 11.0003 - val_accuracy: 0.1274
Epoch 4/13
246/246 [==============================] - 23s 94ms/step - loss: 1.4113 - accuracy: 0.4160 - val_loss: 11.2778 - val_accuracy: 0.1328
Epoch 5/13
246/246 [==============================] - 24s 97ms/step - loss: 1.2298 - accuracy: 0.4999 - val_loss: 1.5009 - val_accuracy: 0.4008
Epoch 6/13
246/246 [==============================] - 24s 96ms/step - loss: 1.1078 - accuracy: 0.5529 - val_loss: 1.4012 - val_accuracy: 0.4435
Epoch 7/13
246/246 [==============================] - 22s 90ms/step - loss: 0.9737 - accuracy: 0.6074 - val_loss: 1.4437 - val_accur

## 